https://stackoverflow.com/questions/43101497/how-do-stream-data-to-a-bokeh-plot-in-jupyter-with-a-high-refresh-rate

Below, makes incremental updates using streaming to ColumnDataSource.
Has disadvantage in blocking subsequent cells from executing.


In [3]:
import time
import numpy as np
# ------------------- new cell ---------------------#

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
# ------------------- new cell ---------------------#

output_notebook()
# ------------------- new cell ---------------------#

my_figure = figure(plot_width=800, plot_height=400)
test_data = ColumnDataSource(data=dict(x=[0], y=[0]))
line = my_figure.line("x", "y", source=test_data)
handle = show(my_figure, notebook_handle=True)

new_data=dict(x=[0], y=[0])
x = []
y = []

step = 0
step_size = 0.1  # increment for increasing step
max_step = 10  # arbitrary stop point for example
period = .1  # in seconds (simulate waiting for new data)
n_show = 10  # number of points to keep and show
while step < max_step:
    x.append(step)
    y.append(np.random.rand())
    new_data['x'] = x = x[-n_show:]  # prevent filling ram
    new_data['y'] = y = y[-n_show:]  # prevent filling ram

    test_data.stream(new_data, n_show)

    push_notebook(handle=handle)
    step += step_size
    time.sleep(period)

Loading BokehJS ...

This moves things nicely to a background thread.

In [5]:
import time
import numpy as np
# ------------------- new cell ---------------------#

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
# ------------------- new cell ---------------------#

output_notebook()
# ------------------- new cell ---------------------#

my_figure = figure(plot_width=800, plot_height=400)
test_data = ColumnDataSource(data=dict(x=[0], y=[0]))
line = my_figure.line("x", "y", source=test_data)
handle = show(my_figure, notebook_handle=True)
# ------------------- new cell ---------------------#

from threading import Thread

stop_threads = False

def blocking_callback(id, stop):
    new_data=dict(x=[0], y=[0])

    step = 0
    step_size = 0.1  # increment for increasing step
    max_step = 10  # arbitrary stop point for example
    period = .1  # in seconds (simulate waiting for new data)
    n_show = 10  # number of points to keep and show

    while True:

        new_data['x'] = [step]
        new_data['y'] = [np.random.rand()]

        test_data.stream(new_data, n_show)

        push_notebook(handle=handle)
        step += step_size
        time.sleep(period)

        if stop():
            print("exit")
            break

thread = Thread(target=blocking_callback, args=(id, lambda: stop_threads))
thread.start()

Loading BokehJS ...

In [6]:
# preceding streaming is not blocking
for cnt in range(10):
    print("Do this, while plot is still streaming", cnt)

# ------------------- new cell ---------------------#

# you might also want to stop the thread
stop_threads=True
del thread

Do this, while plot is still streaming 0
Do this, while plot is still streaming 1
Do this, while plot is still streaming 2
Do this, while plot is still streaming 3
Do this, while plot is still streaming 4
Do this, while plot is still streaming 5
Do this, while plot is still streaming 6
Do this, while plot is still streaming 7
Do this, while plot is still streaming 8
Do this, while plot is still streaming 9
exit
